In [ ]:
from PIL import Image, ImageOps
import os
from rembg import remove
import pandas as pd
import numpy as np
import cv2
import os
import csv
from skimage.feature import graycomatrix, graycoprops
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sb

### Prprocessing Citra

In [ ]:
# os.makedirs("Kopra_Resize", exist_ok=True)
# os.makedirs("Kopra_Resize/Kopra Edible", exist_ok=True)
# os.makedirs("Kopra_Resize/Kopra Reguler", exist_ok=True)
# os.makedirs("Kopra_Resize/Kopra Reject", exist_ok=True)

# for folder in os.listdir("Kopra"):
#     for filename in os.listdir(os.path.join("Kopra", folder)):
#         img = Image.open(os.path.join("Kopra", folder, filename))
#         img = img.resize((800, 600))
#         img.save(os.path.join("Kopra_Resize", folder, filename))
#         print("Image resized to:", os.path.join("Kopra_Resize", folder, filename))

In [ ]:
# os.makedirs("Kopra_Rembg", exist_ok=True)
# os.makedirs("Kopra_Rembg/Kopra Edible", exist_ok=True)
# os.makedirs("Kopra_Rembg/Kopra Reguler", exist_ok=True)
# os.makedirs("Kopra_Rembg/Kopra Reject", exist_ok=True)

# for folder in os.listdir("Kopra_Resize"):
#     for filename in os.listdir(os.path.join("Kopra_Resize", folder)):
#         img = Image.open(os.path.join("Kopra_Resize", folder, filename))
#         result = remove(img)
#         filename = filename.replace(".jpg", ".png")
#         result.save(os.path.join("Kopra_Rembg", folder, filename))
#         print("Background image removed on:", os.path.join("Kopra_Rembg", folder, filename))

In [ ]:
# os.makedirs("Kopra_Crop", exist_ok=True)
# os.makedirs("Kopra_Crop/Kopra Edible", exist_ok=True)
# os.makedirs("Kopra_Crop/Kopra Reguler", exist_ok=True)
# os.makedirs("Kopra_Crop/Kopra Reject", exist_ok=True)

# for folder in os.listdir("Kopra_Rembg"):
#     for filename in os.listdir(os.path.join("Kopra_Rembg", folder)):
#         img = Image.open(os.path.join("Kopra_Rembg", folder, filename))
#         img = img.crop(img.getbbox())
#         img.save(os.path.join("Kopra_Crop", folder, filename))
#         print("Image resized to:", os.path.join("Kopra_Crop", folder, filename))

Ekstraksi Fitur

In [ ]:
path = 'D:/! Reset Pak Husni Teja Sukmana/2023 - Klasifikasi Kopra metode KNN/Implementasi-KNN/dataset/Kopra_Crop/Kopra Edible'
data = os.listdir(path)

rataR = []
rataG = []
rataB = []
rataH = []
rataS = []
rataV = []
ratagray = []
stand = []
luas = []
perimeter = []
contrast = []
dissimilarity = []
homogeneity = []
energy = []
correlation = []
label = []

for gbr in data:
    gbr_read = cv2.imread(os.path.join(path, gbr))
    gbr_rgb = cv2.cvtColor(gbr_read, cv2.COLOR_BGR2RGB)
    (R, G, B) = cv2.split(gbr_rgb)
    meanR = np.mean(R)
    rataR.append(meanR)
    meanG = np.mean(G)
    rataG.append(meanG)
    meanB = np.mean(B)
    rataB.append(meanB)

    gbr_hsv = cv2.cvtColor(gbr_read, cv2.COLOR_BGR2HSV)
    H = gbr_hsv[:, :, 0]
    S = gbr_hsv[:, :, 1]
    V = gbr_hsv[:, :, 2]
    meanH = np.mean(H)
    rataH.append(meanH)
    meanS = np.mean(S)
    rataS.append(meanS)
    meanV = np.mean(V)
    rataV.append(meanV)

    gbr_gray = cv2.cvtColor(gbr_read, cv2.COLOR_BGR2GRAY)
    rata = np.mean(gbr_gray)
    ratagray.append(rata)
    standar = np.std(gbr_gray)
    stand.append(standar)


    # Ekstraksi Fitur Bentuk
    _, thresh = cv2.threshold(gbr_gray, 127, 255, 0)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(contour)
        perimeter_val = cv2.arcLength(contour, True)
    else:
        area = 0
        perimeter_val = 0
    
    luas.append(area)
    perimeter.append(perimeter_val)

    # Ekstraksi Fitur Tekstur (GLCM)
    glcm = graycomatrix(gbr_gray, [1], [0], 256, symmetric=True, normed=True)
    contrast_val = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity_val = graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity_val = graycoprops(glcm, 'homogeneity')[0, 0]
    energy_val = graycoprops(glcm, 'energy')[0, 0]
    correlation_val = graycoprops(glcm, 'correlation')[0, 0]
    
    contrast.append(contrast_val)
    dissimilarity.append(dissimilarity_val)
    homogeneity.append(homogeneity_val)
    energy.append(energy_val)
    correlation.append(correlation_val)

    Jenis_Kopra = '1'
    label.append(Jenis_Kopra)

data1 = pd.DataFrame(rataR, columns=['Mean-R'])
data2 = pd.DataFrame(rataG, columns=['Mean-G'])
data3 = pd.DataFrame(rataB, columns=['Mean-B'])
data4 = pd.DataFrame(rataH, columns=['Mean-H'])
data5 = pd.DataFrame(rataV, columns=['Mean-V'])
data6 = pd.DataFrame(rataS, columns=['Mean-S'])
data7 = pd.DataFrame(ratagray, columns=['Mean-Gray'])
data8 = pd.DataFrame(stand, columns=['Standar-Deviasi'])
data9 = pd.DataFrame(luas, columns=['Luas'])
data10 = pd.DataFrame(perimeter, columns=['Perimeter'])
data11 = pd.DataFrame(contrast, columns=['Contrast'])
data12 = pd.DataFrame(dissimilarity, columns=['Dissimilarity'])
data13 = pd.DataFrame(homogeneity, columns=['Homogeneity'])
data14 = pd.DataFrame(energy, columns=['Energy'])
data15 = pd.DataFrame(correlation, columns=['Correlation'])
data16 = pd.DataFrame(label, columns=['Jenis Kopra'])

listdata1 = [data1, data2, data3, data4, data5, data6, data7, data8, data9, 
             data10, data11, data12, data13, data14, data15, data16]
gabung1 = pd.concat(listdata1, axis=1)


In [ ]:
gabung1

In [ ]:
path = 'D:/! Reset Pak Husni Teja Sukmana/2023 - Klasifikasi Kopra metode KNN/Implementasi-KNN/dataset/Kopra_Crop/Kopra Reguler/'
data = os.listdir(path)

rataR = []
rataG = []
rataB = []
rataH = []
rataS = []
rataV = []
ratagray = []
stand = []
luas = []
perimeter = []
contrast = []
dissimilarity = []
homogeneity = []
energy = []
correlation = []
label = []

for gbr in data:
    gbr_read = cv2.imread(os.path.join(path, gbr))
    gbr_rgb = cv2.cvtColor(gbr_read, cv2.COLOR_BGR2RGB)
    (R, G, B) = cv2.split(gbr_rgb)
    meanR = np.mean(R)
    rataR.append(meanR)
    meanG = np.mean(G)
    rataG.append(meanG)
    meanB = np.mean(B)
    rataB.append(meanB)

    gbr_hsv = cv2.cvtColor(gbr_read, cv2.COLOR_BGR2HSV)
    H = gbr_hsv[:, :, 0]
    S = gbr_hsv[:, :, 1]
    V = gbr_hsv[:, :, 2]
    meanH = np.mean(H)
    rataH.append(meanH)
    meanS = np.mean(S)
    rataS.append(meanS)
    meanV = np.mean(V)
    rataV.append(meanV)

    gbr_gray = cv2.cvtColor(gbr_read, cv2.COLOR_BGR2GRAY)
    rata = np.mean(gbr_gray)
    ratagray.append(rata)
    standar = np.std(gbr_gray)
    stand.append(standar)


    # Ekstraksi Fitur Bentuk
    _, thresh = cv2.threshold(gbr_gray, 127, 255, 0)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(contour)
        perimeter_val = cv2.arcLength(contour, True)
    else:
        area = 0
        perimeter_val = 0
    
    luas.append(area)
    perimeter.append(perimeter_val)

    # Ekstraksi Fitur Tekstur (GLCM)
    glcm = graycomatrix(gbr_gray, [1], [0], 256, symmetric=True, normed=True)
    contrast_val = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity_val = graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity_val = graycoprops(glcm, 'homogeneity')[0, 0]
    energy_val = graycoprops(glcm, 'energy')[0, 0]
    correlation_val = graycoprops(glcm, 'correlation')[0, 0]
    
    contrast.append(contrast_val)
    dissimilarity.append(dissimilarity_val)
    homogeneity.append(homogeneity_val)
    energy.append(energy_val)
    correlation.append(correlation_val)

    Jenis_Kopra = '2'
    label.append(Jenis_Kopra)

data1 = pd.DataFrame(rataR, columns=['Mean-R'])
data2 = pd.DataFrame(rataG, columns=['Mean-G'])
data3 = pd.DataFrame(rataB, columns=['Mean-B'])
data4 = pd.DataFrame(rataH, columns=['Mean-H'])
data5 = pd.DataFrame(rataV, columns=['Mean-V'])
data6 = pd.DataFrame(rataS, columns=['Mean-S'])
data7 = pd.DataFrame(ratagray, columns=['Mean-Gray'])
data8 = pd.DataFrame(stand, columns=['Standar-Deviasi'])
data9 = pd.DataFrame(luas, columns=['Luas'])
data10 = pd.DataFrame(perimeter, columns=['Perimeter'])
data11 = pd.DataFrame(contrast, columns=['Contrast'])
data12 = pd.DataFrame(dissimilarity, columns=['Dissimilarity'])
data13 = pd.DataFrame(homogeneity, columns=['Homogeneity'])
data14 = pd.DataFrame(energy, columns=['Energy'])
data15 = pd.DataFrame(correlation, columns=['Correlation'])
data16 = pd.DataFrame(label, columns=['Jenis Kopra'])

listdata1 = [data1, data2, data3, data4, data5, data6, data7, data8, data9, 
             data10, data11, data12, data13, data14, data15, data16]
gabung2 = pd.concat(listdata1, axis=1)


In [ ]:
gabung2

In [ ]:
path = 'D:/! Reset Pak Husni Teja Sukmana/2023 - Klasifikasi Kopra metode KNN/Implementasi-KNN/dataset/Kopra_Crop/Kopra Reject/'
data = os.listdir(path)

rataR = []
rataG = []
rataB = []
rataH = []
rataS = []
rataV = []
ratagray = []
stand = []
luas = []
perimeter = []
contrast = []
dissimilarity = []
homogeneity = []
energy = []
correlation = []
label = []

for gbr in data:
    gbr_read = cv2.imread(os.path.join(path, gbr))
    gbr_rgb = cv2.cvtColor(gbr_read, cv2.COLOR_BGR2RGB)
    (R, G, B) = cv2.split(gbr_rgb)
    meanR = np.mean(R)
    rataR.append(meanR)
    meanG = np.mean(G)
    rataG.append(meanG)
    meanB = np.mean(B)
    rataB.append(meanB)

    gbr_hsv = cv2.cvtColor(gbr_read, cv2.COLOR_BGR2HSV)
    H = gbr_hsv[:, :, 0]
    S = gbr_hsv[:, :, 1]
    V = gbr_hsv[:, :, 2]
    meanH = np.mean(H)
    rataH.append(meanH)
    meanS = np.mean(S)
    rataS.append(meanS)
    meanV = np.mean(V)
    rataV.append(meanV)

    gbr_gray = cv2.cvtColor(gbr_read, cv2.COLOR_BGR2GRAY)
    rata = np.mean(gbr_gray)
    ratagray.append(rata)
    standar = np.std(gbr_gray)
    stand.append(standar)


    # Ekstraksi Fitur Bentuk
    _, thresh = cv2.threshold(gbr_gray, 127, 255, 0)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(contour)
        perimeter_val = cv2.arcLength(contour, True)
    else:
        area = 0
        perimeter_val = 0
    
    luas.append(area)
    perimeter.append(perimeter_val)

    # Ekstraksi Fitur Tekstur (GLCM)
    glcm = graycomatrix(gbr_gray, [1], [0], 256, symmetric=True, normed=True)
    contrast_val = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity_val = graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity_val = graycoprops(glcm, 'homogeneity')[0, 0]
    energy_val = graycoprops(glcm, 'energy')[0, 0]
    correlation_val = graycoprops(glcm, 'correlation')[0, 0]
    
    contrast.append(contrast_val)
    dissimilarity.append(dissimilarity_val)
    homogeneity.append(homogeneity_val)
    energy.append(energy_val)
    correlation.append(correlation_val)

    Jenis_Kopra = '3'
    label.append(Jenis_Kopra)

data1 = pd.DataFrame(rataR, columns=['Mean-R'])
data2 = pd.DataFrame(rataG, columns=['Mean-G'])
data3 = pd.DataFrame(rataB, columns=['Mean-B'])
data4 = pd.DataFrame(rataH, columns=['Mean-H'])
data5 = pd.DataFrame(rataV, columns=['Mean-V'])
data6 = pd.DataFrame(rataS, columns=['Mean-S'])
data7 = pd.DataFrame(ratagray, columns=['Mean-Gray'])
data8 = pd.DataFrame(stand, columns=['Standar-Deviasi'])
data9 = pd.DataFrame(luas, columns=['Luas'])
data10 = pd.DataFrame(perimeter, columns=['Perimeter'])
data11 = pd.DataFrame(contrast, columns=['Contrast'])
data12 = pd.DataFrame(dissimilarity, columns=['Dissimilarity'])
data13 = pd.DataFrame(homogeneity, columns=['Homogeneity'])
data14 = pd.DataFrame(energy, columns=['Energy'])
data15 = pd.DataFrame(correlation, columns=['Correlation'])
data16 = pd.DataFrame(label, columns=['Jenis Kopra'])

listdata1 = [data1, data2, data3, data4, data5, data6, data7, data8, data9, 
             data10, data11, data12, data13, data14, data15, data16]
gabung3 = pd.concat(listdata1, axis=1)


In [ ]:
display(pd.concat([gabung1, gabung2, gabung3], ignore_index = True))

In [ ]:

listdata = [gabung1, gabung2, gabung3]
total = pd.concat(listdata, ignore_index=True)
total.to_csv("D:/ekstrasi-fitur-kopra-nonTelungkup.csv", index=False)
total.to_excel("D:/ekstrasi-fitur-kopra-nonTelungkup.xlsx", index=False)

print("tahap preprocessing telah selsai dibuat")

Pengolahan Model untuk Klasifikasi

In [ ]:
#menampung dataset untuk warna, bentuk, dan tekstur
def_Kopra = pd.read_csv('D:/ekstrasi-fitur-kopra-nonTelungkup.csv')
#dataset warna dan tekstur
#def_Kopra = pd.read_csv('ekstrasi-fitur-kopra-nonTelungkup1.csv')

Split Dependent (Target/Label) Independent (atribut) Value

In [ ]:
#skenario W
#x = def_Kopra.iloc[:,0:8].values
#y = def_Kopra.iloc[:,[-1]].values

#skenario B
#x = def_Kopra.iloc[:,8:10].values
#y = def_Kopra.iloc[:,[-1]].values

#skenario T
#x = def_Kopra.iloc[:,10:15].values
#y = def_Kopra.iloc[:,[-1]].values

#Skenario W dan B
#x = def_Kopra.iloc[:,0:10].values
#y = def_Kopra.iloc[:,[-1]].values

#Skenario W dan T
#x = def_Kopra.iloc[:,0:13].values
#y = def_Kopra.iloc[:,[-1]].values

#Skenario B dan T
x = def_Kopra.iloc[:,8:15].values
y = def_Kopra.iloc[:,[-1]].values

# #Skenario W,B,dan T
# x = def_Kopra.iloc[:,0:15].values
# y = def_Kopra.iloc[:,[-1]].values

In [ ]:
y = y.reshape(len(y))

Feature Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x = scaler.fit_transform(x)

Split Data Testing dan Data Training

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.1,random_state=0)

Klasifikasi dengan KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3,metric='euclidean').fit(x_train,y_train)
knn

Prediksi

In [ ]:
y_prediksi = knn.predict(x_test)

Akurasi

In [ ]:
#Metrik Akurasi
# Membuat DataFrame untuk data uji
df_prediksi = pd.DataFrame({'Prediksi': y_prediksi, 'Label Sebenarnya': y_test})

# Mencetak citra yang terprediksi benar dan salah
for index, row in df_prediksi.iterrows():
    if row['Prediksi'] == row['Label Sebenarnya']:
        print(f"Citra ke-{index+1}: Prediksi benar")
    else:
        print(f"Citra ke-{index+1}: Prediksi salah")

# Menghitung jumlah prediksi yang benar
jumlah_benar = (df_prediksi['Prediksi'] == df_prediksi['Label Sebenarnya']).sum()
jumlah_seluruh = len(df_prediksi)

# Menghitung dan mencetak nilai akurasi
nilai_akurasi = jumlah_benar / jumlah_seluruh
print("Jumlah prediksi yang benar:", jumlah_benar)
print("Jumlah seluruh prediksi:", jumlah_seluruh)
print("Nilai akurasi:", nilai_akurasi)


In [ ]:
#confusion matrix dan classificatio report
print(classification_report(y_test, y_prediksi))
sb.heatmap(confusion_matrix(y_test, y_prediksi),annot=True)

In [ ]:
n_akurasi = []
for k in range (1,30):
    knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean').fit(x_train,y_train)
    y_prediksi = knn.predict(x_test)
    akurasi = accuracy_score(y_test,y_prediksi)
    n_akurasi.append(akurasi)
print(n_akurasi)